In [150]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [5]:
import pandas as pd  # provides interface for interacting with tabular data
import geopandas as gpd  # combines the capabilities of pandas and shapely for geospatial operations
from shapely.geometry import Point, Polygon, MultiPolygon  # for manipulating text data into geospatial shapes
from shapely import wkt  # stands for "well known text," allows for interchange across GIS programs
import rtree  # supports geospatial join
import os
import fnmatch
import numpy as np
import matplotlib.pyplot as plt
import descartes
import sys
import sklearn
from shapely.ops import nearest_points
from datetime import datetime as dt, date
sys.path.append('/Users/saraprice/Documents/NYU/Fall_2020/DS_GA_1001/final_project/wildfires-1001/code/functions/')
from gis_processing import *
import pickle

In [2]:
data_dir = '/Users/saraprice/Documents/NYU/Fall_2020/DS_GA_1001/final_project/wildfires-1001/data'

In [32]:
target_df = pd.read_pickle(os.path.join(data_dir, 'clean_data/target_df_final_geo.pkl'))
weather_dict_path = os.path.join(data_dir, 'clean_data/ERA_weather-data/ERA_rename_dictionary.pkl')

In [18]:
with open(weather_dict_path, 'rb') as handle:
    rename_dict = pickle.load(handle)

In [26]:
target_df.rename(columns = rename_dict, inplace = True)

In [133]:
non_mod_cols = ['GRID_ID','month_id','MONTH','COUNTYFP','NAME','GRID_AREA','COUNTY_ARE','month_id_old_x','month_id_old_y',
                'geometry', 'adj_fire_count','adj_fire_bcount']
Y_cols = ['Y_bin', 'Y_fire_count', 'Y_fire_area_prop', 'Y_fire_class_size']

In [127]:
non_weather_cols = [i for i in train_data.columns if str.find(i,"hrs")==-1]
weather_cols = [i for i in train_data.columns if i not in(non_weather_cols)]
oth_subset = ['elev_mean','elev_std','elev_media','elev_max','elev_min','elev_range','adj_fire_count','adj_fire_bcount']

In [141]:
sub = target_df[['YEAR', 'adj_fire_count','adj_fire_bcount']]
sub[sub['adj_fire_count'].isnull()]['YEAR'].value_counts()

2019    1188
Name: YEAR, dtype: int64

In [86]:
for col in target_df.columns:
    if target_df[col].dtypes == 'float64':
        target_df[col] = target_df[col].astype(np.float32)

## Train- test Split

In [144]:
train_data = target_df[target_df['YEAR']<=2016]
X_train = train_data.drop('YEAR', axis = 1)
for y in Y_cols + non_mod_cols:
    try:
        X_train.drop(y, inplace = True, axis =1)
    except:
        pass
Y_train = train_data[['Y_fire_class_size']]

In [146]:
test_data = target_df[target_df['YEAR']>2016]
X_test = test_data.drop('YEAR', axis = 1)
for y in Y_cols + non_mod_cols:
    try:
        X_test.drop(y, inplace = True, axis =1)
    except:
        pass
Y_test = test_data[['Y_fire_class_size']] 

In [148]:
X_test.columns.tolist()

['elev_mean',
 'elev_std',
 'elev_media',
 'elev_max',
 'elev_min',
 'elev_range',
 'pl_count',
 'total_pl_length',
 'road_count',
 'total_road_length',
 'MEDIAN_AGE_TOT',
 'MEDIAN_AGE_MALE',
 'MEDIAN_AGE_FEM',
 'AGEUNDER13_TOT',
 'AGE1424_TOT',
 'AGE2544_TOT',
 'AGE4564_TOT',
 'AGE65PLUS_TOT',
 'COUNTY_AREA',
 'POPDENSITY',
 'POPDENSITY_MALE',
 'POPDENSITY_FEM',
 'Structure Arsons',
 'Mobile Arsons',
 'Other Arsons',
 'Total Arsons',
 'Total Arsons Cleared',
 'Unemployment',
 'medianHHI2018',
 '10 metre U wind component hrs:0',
 '10 metre V wind component hrs:0',
 '2 metre dewpoint temperature hrs:0',
 '2 metre temperature hrs:0',
 'Leaf area index, high vegetation hrs:0',
 'Leaf area index, low vegetation hrs:0',
 'Surface pressure hrs:0',
 'Total precipitation hrs:0',
 '10 metre U wind component hrs:6',
 '10 metre V wind component hrs:6',
 '2 metre dewpoint temperature hrs:6',
 '2 metre temperature hrs:6',
 'Leaf area index, high vegetation hrs:6',
 'Leaf area index, low vegetation 

In [147]:
rf_clf = RandomForestClassifier().fit(X_train, Y_train)

<ipython-input-147-ba37c60503a8>:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_clf = RandomForestClassifier().fit(X_train, Y_train)


In [155]:
y_preds_class_size = rf_clf.predict(X_test)

In [163]:
y_eval.shape

(3564, 1)

In [168]:
tab = dict(zip([0,1,2,3,4],[0,1,2,3,4]))
y_eval = np.vectorize(tab.get)(Y_test)
y_eval = y_eval.reshape(3564)
y_pred = np.vectorize(tab.get)(y_preds_class_size)
# Making the Confusion Matrix
print(pd.crosstab(y_eval, y_pred, rownames=['Actual Classes'], colnames=['Predicted Classes']))

Predicted Classes     0   1   2   3   4
Actual Classes                         
0                  2910  38  10  22  30
1                   203  32   4  23  16
2                   109   5   7   3   4
3                    56   4   2   2   2
4                    66   5   4   5   2


In [173]:
compare_df = Y_test.copy()
compare_df['Y_class_size_pred'] = y_preds_class_size
len(compare_df[(compare_df['Y_class_size_pred']==compare_df['Y_fire_class_size']) & (compare_df['Y_fire_class_size']>0)])

43

In [178]:
len(compare_df[(compare_df['Y_class_size_pred']>0) & (compare_df['Y_fire_class_size']==0)])/554

0.18050541516245489